In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import AgglomerativeClustering
import copy

In [19]:
data_B_1 = pd.read_csv('dataB1.csv')
data_B_2 = pd.read_csv('dataB2.csv')
data_B_3 = pd.read_csv('dataB3.csv')
data_B_4 = pd.read_csv('dataB4.csv')
data_B_5 = pd.read_csv('dataB5.csv')
data_B_1_len = data_B_1.shape[0]
data_B_2_len = data_B_2.shape[0]
data_B_3_len = data_B_3.shape[0]
data_B_4_len = data_B_4.shape[0]
data_B_5_len = data_B_5.shape[0]

MAX_ITEM_NUM = 1000
MAX_ITEM_AREA = 250000000

In [20]:
data_B_1.head()

,item_id,item_material,item_num,item_length,item_width,item_order
0,0,YW1-0218S,1,858.0,337.5,order539
1,1,YW1-0218S,1,425.5,192.0,order317
2,2,YW1-0218S,1,195.0,48.0,order317
3,3,YW1-0215S,1,302.0,480.0,order392
4,4,YW1-0215S,1,1022.0,330.0,order227


In [21]:
# raw_item_material_test = list(data_B_1.iloc[:,1])
# print(raw_item_material_test)

In [22]:
def data_pre(data_frame, data_len):
    raw_item_material_list = list(data_frame.iloc[:,1])
    data_frame['raw_material'] = raw_item_material_list
    data_frame.iloc[:,1] = LabelEncoder().fit_transform(data_frame.iloc[:,1])
    data_area_list = []
    data_index_list = []
    order_list = []
    for i in range(data_len):
        data_frame['item_order'][i] = int(str(data_frame['item_order'][i].strip('order')))
        data_length = data_frame['item_length'][i]
        data_width = data_frame['item_width'][i]
        data_area_list.append(data_length * data_width)
        data_index_list.append(i)
        now_order = data_frame['item_order'][i]
        if now_order not in order_list:
            order_list.append(now_order)
    data_frame['Area'] = data_area_list
    data_frame['Index'] = data_index_list
    # data_frame.sort_values(by="Area")
    return order_list

In [23]:
B1_order = data_pre(data_B_1, data_B_1_len)
B2_order = data_pre(data_B_2, data_B_2_len)
B3_order = data_pre(data_B_3, data_B_3_len)
B4_order = data_pre(data_B_4, data_B_4_len)
B5_order = data_pre(data_B_5, data_B_5_len)

d:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [24]:
# data_B_1.head()
# data_B_1['item_order'][0]
# data_B_1['item_material'][0]

In [25]:
# 根据订单号分类
B1_order_group = data_B_1.groupby(['item_order'])
B2_order_group = data_B_2.groupby(['item_order'])
B3_order_group = data_B_3.groupby(['item_order'])
B4_order_group = data_B_4.groupby(['item_order'])
B5_order_group = data_B_5.groupby(['item_order'])

In [26]:
class S_Cluster:
    # 输入生成簇的数据集，id号，总数据集的长度
    def __init__(self, data_frame, cluster_id):
        self.data_frame = data_frame
        self.item_num_sum = data_frame.shape[0]
        self.item_area_sum = sum(list(data_frame['Area']))
        self.material_set = set(data_frame['item_material'])
        self.cluster_id = cluster_id
        self.cluster_id_list = list(set(data_frame['item_order']))
    # def apped_cluster(extern_cluster):

In [27]:
test_data_frame_0 = B1_order_group.get_group(1)
test_data_frame_1 = B1_order_group.get_group(2)
test_data_frame_2 = pd.concat([test_data_frame_0, test_data_frame_1])

In [28]:
B1_order_S_cluster_list = []
j = 0
for i in B1_order:
    B1_order_S_cluster_list.append(S_Cluster(B1_order_group.get_group(i), j))
    j = j + 1

In [29]:
def get_distance(cluster_list):
    data_len = len(cluster_list)
    dist_mat = np.eye(data_len)
    for i in range(data_len):
        for j in range(i + 1, data_len):
            i_material_set = cluster_list[i].material_set
            j_material_set = cluster_list[j].material_set
            U_set = i_material_set | j_material_set
            N_set = i_material_set & j_material_set
            # 越相似距离越小，完全一样为0
            # 完全不一样为1
            set_dist = (len(U_set) - len(N_set)) / len(U_set)
            dist_mat[i][j] = dist_mat[j][i] = set_dist
    return dist_mat
    

In [30]:
B1_raw_dist_mat = get_distance(B1_order_S_cluster_list)
print(B1_raw_dist_mat)
print(B1_raw_dist_mat[507][503])

[[1.  0.6 0.5 ... 1.  1.  1. ]
 [0.6 1.  0.6 ... 1.  1.  1. ]
 [0.5 0.6 1.  ... 1.  1.  1. ]
 ...
 [1.  1.  1.  ... 1.  0.5 1. ]
 [1.  1.  1.  ... 0.5 1.  1. ]
 [1.  1.  1.  ... 1.  1.  1. ]]
0.25


In [31]:
def get_cluster(small_cluster_list, dist_mat, max_dist):
    now_min_dist = np.min(dist_mat)
    temp_cluster_list = copy.deepcopy(small_cluster_list)
    dist_mat = copy.deepcopy(dist_mat)
    # 用于给新生成的小聚类编号
    new_cluster_id = temp_cluster_list[len(temp_cluster_list) - 1].cluster_id + 1
    while now_min_dist < max_dist:   
        print()
        now_min_dist = np.min(dist_mat)
        np_min_index = np.argmin(dist_mat)
        print("now_min: " + str(now_min_dist))
        y_index = int(np_min_index / len(temp_cluster_list))
        x_index = np_min_index % len(temp_cluster_list)
        print("get_x_index: " + str(x_index))
        print("get_y_index: " + str(y_index))
        x_cluster = temp_cluster_list[x_index]
        y_cluster = temp_cluster_list[y_index] 
        print("x_cluster_id: " + str(x_cluster.cluster_id))
        print("y_cluster_id: " + str(y_cluster.cluster_id))
        # 如果两者满足条件，则可合并为一个聚类
        if x_cluster.item_area_sum + y_cluster.item_area_sum <= MAX_ITEM_AREA and x_cluster.item_num_sum + y_cluster.item_num_sum <= MAX_ITEM_NUM:
            new_cluster_data_frame = pd.concat([x_cluster.data_frame, y_cluster.data_frame])
            temp_cluster_list.pop(x_index)
            temp_cluster_list.pop(y_index)
            new_cluster = S_Cluster(new_cluster_data_frame, new_cluster_id)
            print("由" + str(x_cluster.cluster_id) + " 和 " + str(y_cluster.cluster_id) + " 聚类成 " + str(new_cluster_id))
            temp_cluster_list.append(new_cluster)
            new_cluster_id = new_cluster_id + 1
            dist_mat = get_distance(temp_cluster_list)
            # print(dist_mat)
            print("End_num" + str(len(temp_cluster_list)))
        else:
            print("不满足合并条件")
            dist_mat[x_index][y_index] = 1
            dist_mat[y_index][x_index] = 1
        if(len(temp_cluster_list) == 1):
            return temp_cluster_list
    return temp_cluster_list


In [32]:
B1_cluster_list = get_cluster(B1_order_S_cluster_list, B1_raw_dist_mat, 1)
# B1_order_S_cluster_list[0]


now_min: 0.0
get_x_index: 31
get_y_index: 0
x_cluster_id: 31
y_cluster_id: 0
由31 和 0 聚类成 546
End_num545

now_min: 0.0
get_x_index: 16
get_y_index: 0
x_cluster_id: 17
y_cluster_id: 1
由17 和 1 聚类成 547
End_num544

now_min: 0.0
get_x_index: 56
get_y_index: 0
x_cluster_id: 60
y_cluster_id: 2
由60 和 2 聚类成 548
End_num543

now_min: 0.0
get_x_index: 7
get_y_index: 1
x_cluster_id: 10
y_cluster_id: 4
由10 和 4 聚类成 549
End_num542

now_min: 0.0
get_x_index: 17
get_y_index: 6
x_cluster_id: 23
y_cluster_id: 11
由23 和 11 聚类成 550
End_num541

now_min: 0.0
get_x_index: 13
get_y_index: 7
x_cluster_id: 20
y_cluster_id: 13
由20 和 13 聚类成 551
End_num540

now_min: 0.0
get_x_index: 13
get_y_index: 7
x_cluster_id: 22
y_cluster_id: 14
由22 和 14 聚类成 552
End_num539

now_min: 0.0
get_x_index: 35
get_y_index: 7
x_cluster_id: 48
y_cluster_id: 15
由48 和 15 聚类成 553
End_num538

now_min: 0.0
get_x_index: 71
get_y_index: 8
x_cluster_id: 87
y_cluster_id: 18
由87 和 18 聚类成 554
End_num537

now_min: 0.0
get_x_index: 11
get_y_index: 8
x

In [33]:
# LabelEncoder.inverse_transform(list(B1_cluster_list[0].data_frame['item_material']))

In [53]:
data_B_1.head()
material_list = list(set(data_B_1['raw_material']))
material_dict = {'material':material_list}
print(material_list)
df_material = pd.DataFrame(material_dict)
df_material.to_csv("B1_material.csv")

['5-0218SD', 'XMJQ-0215S', 'CGS-0218S', 'XYHX-0218S', 'YGKQ-0418S', 'QKQ-0225S', 'GDMY-0218SD', 'PGBL-0418S', 'MNH-0225S', 'ZYF-0218SD', 'QSYM-0215S', 'XXB-0218SD', 'SRBW-0215S', 'YDBS-0225S', 'YW1-0235S', 'NHQL-0218S', 'FMB-0235S', 'XYH-0215S', '5-0218S', 'GDMY-0225S', 'LMW-0215S', 'YW10-0218S', 'LMW-0225S', 'ALJQ-0218SD', 'NJYH-0225S', 'QSYM-0225S', 'NJHL-0215S', 'YMH-0215S', 'NBSY-0215S', 'YTM-0225S', 'ALJQ-0215S', 'HT9-0225S', 'ZZL-0218S', 'LMW-0218SD', 'MNH-0218SD', 'GHSS-0215S', 'CBZP-0218S', 'QKQ-0218SD', 'NJYH-0218SD', 'NBSY-0235S', 'YW1-0225S', 'YDBS-0218SD', 'NJYH-0218S', 'YMH-0218SD', 'GDMY-0215S', 'YW8-0218SD', 'CGS-0218SD', 'NDSF-0218S', 'GHSS-0218S', 'YW10-0225S', 'NJBB-0215S', 'JH-0215S', 'SL-0218S', 'XXB-0225S', 'PDLL-0418S', 'GHSS-0218S-D', 'YTM-0215S', 'FMB-0218S', 'YSH-0218SD', '5-0215S', 'JH-0218S', 'YGKQ-0418SD', 'PKQL-0418S', 'HT9-0218S', 'QKQ-0215S', 'FMB-0215S', 'YW10-0215S', 'XMJQ-0218SD', 'YDBS-0215S', 'GDMY-0225SD', 'YSH-0218S', 'XYHX-0225S', 'YHXM-0218S', 'F

In [55]:
exam_list = []

for i in range(len(B1_cluster_list)):
    print()
    print("第" + str(i) + "批包含以下订单")
    print(B1_cluster_list[i].cluster_id_list)
    exam_list.extend(B1_cluster_list[i].cluster_id_list)
    temp_cluster_data_frame = B1_cluster_list[i].data_frame
    temp_cluster_data_frame.to_csv(str(i) + "_B1_" + "_cluster.csv")
    temp_cluster_material_list = list(set(list(temp_cluster_data_frame['item_material'])))
    temp_cluster_data_frame_group = temp_cluster_data_frame.groupby('item_material')
    for j in temp_cluster_material_list:
        out_put_data_frame = temp_cluster_data_frame_group.get_group(j)
        out_put_data_frame.to_csv(str(i) + "_B1_"+ str(j) + "_cluster.csv")
exam_list.sort()
# exam_list.sort()
# print(exam_list)



第0批包含以下订单
[144, 142, 143]

第1批包含以下订单
[292, 12, 205, 206, 207, 13, 209, 146, 208, 204]

第2批包含以下订单
[353, 258, 257, 358, 359, 6, 361, 263, 241, 437, 92, 93]

第3批包含以下订单
[64, 1, 2, 374, 281, 282, 283, 60]

第4批包含以下订单
[186, 187, 188]

第5批包含以下订单
[418, 357, 487, 488, 362, 236, 439, 441]

第6批包含以下订单
[352, 384, 260, 295, 296, 297, 298, 137, 381]

第7批包含以下订单
[162, 165, 396, 398, 402, 179, 18, 19, 506, 508, 509]

第8批包含以下订单
[97, 99, 364, 365, 366, 367, 16, 369, 440, 219, 252]

第9批包含以下订单
[69, 429, 271, 272, 273, 274, 51, 276, 53, 54, 52, 275, 475]

第10批包含以下订单
[417, 289, 420, 453, 50, 115, 20, 309, 409, 123, 286, 510, 415]

第11批包含以下订单
[514, 223, 516, 515, 135, 71, 424, 139, 141, 174, 310, 350, 351]

第12批包含以下订单
[65, 66, 323, 324, 326, 327, 328, 462, 375]

第13批包含以下订单
[472, 211, 212, 280, 473, 349]

第14批包含以下订单
[546, 45, 46, 47]

第15批包含以下订单
[10, 537, 410, 411, 150, 55, 152, 153, 154, 155, 156, 413, 412]

第16批包含以下订单
[158, 145, 157, 151, 376, 538, 88, 444, 445, 446, 447]

第17批包含以下订单
[136, 232, 457, 524, 493,

In [ ]:
# writer=pd.ExcelWriter('E:\\text.xlsx')
